# 协程

# 迭代器
- 可迭代:直接作用与for循环的变量
- 迭代器:不但可以作用于for循环,还可以被nxet调用
- list 是个典型的可迭代对象,但不是迭代器
- 通过isinstance判断
- iterable和iterator可以相互转换
    - 通过iter函数

In [2]:
# 可迭代
l = [i for i in range(10)]

for a in l:
    print(a)
    

    
# range 是个迭代器
for i in range(5):
    print(i)


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4


In [6]:
# isinstance案例
# 判断某个变量是否是一个实例

# 判断是否可迭代

from collections import Iterable
ll = [1,2,3,4,5]
print(isinstance(ll, Iterable))

from collections import Iterator
print(isinstance(ll, Iterator))

True
False


In [9]:
# iter 函数
s = "i love wangxaiojing"

print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)
print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))


True
False
True
True


# 生成器
- generator: 一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件:
    - 每次调用都产生出for循环需要的下一个元素
    - 如果达到最后一个后,爆出StopIteration异常
    - 可以被next调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield, 则这个函数就叫生成器
    - next调用函数,遇到yield返回


In [1]:
# 直接使用生成器
L = [x*x for x in range(5)]  # 放在中括号里是列表生成器
g = (x*x for x in range(5))  # 放在小括号里是生成器

print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [2]:
# 函数案例

def odd():
    print("Step 1")
    print("Step 2")
    print("Step 3")
    return None

odd()

Step 1
Step 2
Step 3


In [5]:
# 生成器的案例
# 在函数odd中,yield负责返回
def odd():
    print("Step 1")
    yield 1
    print("Step 2")
    yield 2
    print("Step 3")
    yield 3

# odd()是调用生成器 
g = odd()
one = next(g)
print(one)

two = next(g)
print(two)

three = next(g)
print(three)

Step 1
1
Step 2
2
Step 3
3


In [6]:
# for 循环调用生成器
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, b+a
        n += 1 
    
    return 'Done'

fib(5)

1
1
2
3
5


'Done'

In [7]:
# 斐波那契数列的生成器的写法
def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, b+a
        n += 1 
    
    return 'Done'

g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


TypeError: 'str' object is not an iterator

In [8]:
ge = fib(10)
'''
生成器的典型用法就是在for中使用
比较常用的典型生成器就是 range
'''
for i in ge :
    print(i)

1
1
2
3
5
8
13
21
34
55
D
o
n
e


 # 协程
 - 历史历程
     - 3.4引入协程, 用yield实现
     - 3.5引入协程语法
     - 实现协程比较好的包有 asyncio, tornado, gevent
 - 从技术角度讲,协程就是一个你可以暂停执行的函数,或者干脆把协程理解为生成器
 - 协程的实现
     - yield返回
     - send调用
- 协程的四个状态
    - 详见 课程笔记
- 协程终止
    - 协程中未处理的异常会向上冒泡,传给 next 函数或者 send 方法的调用方(即出发协程的对象)
    - 终止协程的一种方式:发送某个哨符值, 让协程退出. 内置的 None 和 Ellipsis 等常量经常用作哨符值==
- yield from
    - 调用协程为了得到返回值,携程必须正常终止
    - 生成器正常终止会发出StopIteration异常,异常对象的value属性保存返回值
    - yield from 从内部捕获StopIteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from表达式的生成器函数
        - 委派生成器在yield from表达式处暂停, 调用方可以直接把数据发给子生成器
        - 子生成器再把产出的值发给调用方
        - 子生成器在最后,解释器会抛出StopIteration, 并把返回值附加到异常对象上
        

In [10]:
# 协程代码
def simple_coroutine():
    print('-> start')
    x = yield
    print('-> recived', x)
    
# 主线程
sc = simple_coroutine()
print(1111)
# 可以使用sc.send(None), 效果一样
next(sc) # 预激

print(2222)
sc.send('zhexiao')

1111
-> start
2222
-> recived zhexiao


StopIteration: 

In [11]:
# 协程的状态

def simple_coroutine(a):
    print('-> start')
    
    b = yield a 
    print('-> recived', a, b)
    
    c = yield a + b
    print('-> recived', a, b, c)
    
# runc
sc = simple_coroutine(5)

aa = next(sc)
print(aa)
bb = sc.send(6) #5, 6
print(bb)
cc = sc.send(7)  #5, 6, 7
print(cc)

-> start
5
-> recived 5 6
11
-> recived 5 6 7


StopIteration: 

In [13]:
# 案例v03

def gen():
    for c in "AB":
        yield c
        
# list 直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from "AB"
    
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [31]:
# 案例v04, 委派生成器
from collections import namedtuple

'''


'''
ResClass = namedtuple('Res', 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count
        
    return ResClass(count, average)

# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()
        
# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }
    
    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)
        
        # 预激协程
        next(coroutine)
        
        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)
            
        # 终止协程
        coroutine.send(None)
    print(storages)
    

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


# asyncio
- python3.4开始引入标准库当中,内置对异步io的支持
- asyncio本身是一个消息循环
- 步骤:
    - 创建消息循环
    - 把携程导入
    - 关闭

In [35]:
import threading
# 引入异步io包
import asyncio

# 使用携程
@asyncio.coroutine
def hello():
    print('hello world!(%s)'%threading.currentThread())
    print('Start......(%s)'%threading.currentThread())
    yield from asyncio.sleep(10)
    print('done......(%s)'%threading.currentThread())
    print('hello again!(%s)'%threading.currentThread())

# 启动消息循环    
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待tasks执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()

hello world!(<_MainThread(MainThread, started 139982593656576)>)
Start......(<_MainThread(MainThread, started 139982593656576)>)
hello world!(<_MainThread(MainThread, started 139982593656576)>)
Start......(<_MainThread(MainThread, started 139982593656576)>)
done......(<_MainThread(MainThread, started 139982593656576)>)
hello again!(<_MainThread(MainThread, started 139982593656576)>)
done......(<_MainThread(MainThread, started 139982593656576)>)
hello again!(<_MainThread(MainThread, started 139982593656576)>)


# async  and  await
- 为了更好的表示异步io
- python3.5引入
- 让携程代码更简洁
- 使用上,可以简单地进行替换
    - 用async替换@asyncio.coroutine
    - await 替换 yield from

In [42]:
# 案例

import threading
# 引入异步io包
import asyncio

# 使用携程
#@asyncio.coroutine
async def hello():
    print('hello world!(%s)'%threading.currentThread())
    print('Start......(%s)'%threading.currentThread())
    await asyncio.sleep(10)
    print('done......(%s)'%threading.currentThread())
    print('hello again!(%s)'%threading.currentThread())

# 启动消息循环    
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待tasks执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()

namedtuple_TokenInfo:14: RuntimeWarning: coroutine 'hello' was never awaited
/var/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: coroutine 'hello' was never awaited


RuntimeError: Event loop is closed

# aiohttp
- asyncio实现单线程的并发io,在客户端用处不大
- 在服务器端可以asyncio+coroutine配合,因为http是io操作
- asyncio实现了tcp, udp, ssl等协议
- aiohttp是给予asyncio实现的http框架
- pip install aiohttp安装

In [43]:
# aiohttp案例

# 见课程笔记

# concurrent.futures
- python3新增的库
- 类似其他语言线程池的概念
- 利用multiprocessiong实现真正的并行计算
- 核心原理: 以子进程的形式,并行运行多个python解释器, 从而令python程序可以利用多核cpu来提升执行速度.
- 由于子进程与主解释器相分离,所以他们的全局解释器锁也是相互独立的.
- 每个子进程都能够完整的使用一个cpu内核
- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
    - 执行的时候需要自行选择
- submit(fn, args, kwargs)
    - fn:异步执行的函数
    - args, kwargs参数

# current中map函数
- map(fn, \*iterables, timeout=None)
    - 跟map函数类似
    - 函数需要异步执行
    - timeout: 超时时间
    
# future
